In [1]:
1+1

2

In [2]:
import os
%pwd

'c:\\Users\\am600\\MLOPS\\text_summarizer\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\am600\\MLOPS\\text_summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [5]:
%pwd

'c:\\Users\\am600\\MLOPS\\text_summarizer'

In [6]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
            
        )

        return model_evaluation_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

import torch
import pandas as pd
from tqdm import tqdm

c:\Users\am600\MLOPS\text_summarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate


class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split list into batch-sized chunks"""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_ds(
        self,
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=4,
        device="cuda" if torch.cuda.is_available() else "cpu",
        column_text="dialogue",
        column_summary="summary"
    ):

        article_batches = list(
            self.generate_batch_sized_chunks(dataset[column_text], batch_size)
        )
        target_batches = list(
            self.generate_batch_sized_chunks(dataset[column_summary], batch_size)
        )

        for article_batch, target_batch in tqdm(
                zip(article_batches, target_batches),
                total=len(article_batches)
        ):

            inputs = tokenizer(
                article_batch,
                max_length=512,              # 🔥 Reduced for speed
                truncation=True,
                padding=True,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():          # 🔥 Important for speed
                summaries = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=64,          # 🔥 Reduced for speed
                    num_beams=2             # 🔥 Reduced beams for speed
                )

            decoded_summaries = tokenizer.batch_decode(
                summaries,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )

            metric.add_batch(
                predictions=decoded_summaries,
                references=target_batch
            )

        score = metric.compute()
        return score

    def evaluate(self):

        device = "cuda" if torch.cuda.is_available() else "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_path
        ).to(device)

        model.eval()  # 🔥 Important

        # Load dataset
        dataset = load_from_disk(self.config.data_path)

        # 🔥 Fast Evaluation Subset (Remove later for full eval)
        dataset["test"] = dataset["test"].select(range(50))

        rouge_metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset["test"],
            rouge_metric,
            model,
            tokenizer,
            batch_size=4
        )

        rouge_dict = {
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLsum": score["rougeLsum"],
        }

        df = pd.DataFrame([rouge_dict])
        df.to_csv(self.config.metric_file_name, index=False)

        print("\nEvaluation Complete ✅")
        print(df)


In [15]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
model_evaluation_config.evaluate()

[2026-02-12 21:49:26,164: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-12 21:49:26,169: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-12 21:49:26,172: INFO: common: created directory at: artifacts]
[2026-02-12 21:49:26,174: INFO: common: created directory at: artifacts/model_evaluation]


Loading weights: 100%|██████████| 682/682 [00:01<00:00, 404.70it/s, Materializing param=model.shared.weight]                                   
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
100%|██████████| 13/13 [04:05<00:00, 18.85s/it]

[2026-02-12 21:53:42,014: INFO: rouge_scorer: Using default tokenizer.]



Evaluation Complete ✅
     rouge1  rouge2    rougeL  rougeLsum
0  0.013718     0.0  0.013381   0.013259
